In [1]:
import numpy as np
import multiprocessing as mp
from multiprocessing.managers import BaseManager
from time import time, sleep
from task_3.src.waypoint2 import Rectangle
class a:
    def __init__(self, name):
        self.name = name
        self.hold = 0
        self.pos = np.array([0,0,0])
        self.desiredWP = np.array([0,0,0])
        self.n = 0

    def setpoint(self, wayPt, deadband_width= 0.1):
        self.desiredWP = wayPt
        self.n = 0
        while self.hold or self.n < 5:
            sleep(1)
            self.pos = self.pos + 0.1
            self.n += 1
            # print(self.name + " pos: ", self.pos)
class testClass:
    def __init__(self):
        self.drone = [a('Drone1'), a('Drone2')]
        self.toggle_state = 0
        self.drone2WP = [[], [[-10, -10, -100], [10,10,100]], []] 
        self.X = np.array([-0.73, 0.8, 0.84, -.75])
        self.Y = np.array([0.45, 0.41, -0.31, -0.31])
        self.Z = 0.5 
        self.genPath()
        print(self.total)
        print(self.drone2WP)

    def genPath(self):
        self.rect = Rectangle(self.X, self.Y, self.Z, 5, 4)
        self.rect.compute_slopes()
        rectWP = self.rect.generate_points()
        self.rectWP1 = rectWP[0:self.rect.S1,:]
        rectWP2 = rectWP[self.rect.S1:(self.rect.S1+self.rect.S2),:]
        rectWP3 = rectWP[(self.rect.S1+self.rect.S2):(self.rect.S1+self.rect.S2+self.rect.S3),:]
        rectWP4 = rectWP[(self.rect.S1+self.rect.S2+self.rect.S3):(self.rect.S1+self.rect.S2+self.rect.S3+self.rect.S4),:]
        self.total = [rectWP2,rectWP3,rectWP4,self.rectWP1]


    def targetDrone1(self):
        for waypts in self.total:
            for wayPt in waypts:
                self.drone[0].setpoint(wayPt)
                self.drone2WP[2].append(wayPt)
                print("waypt reached: Drone1", wayPt, self.toggle_state)
                sleep(1)
            self.toggle_state = 1
            self.drone2WP[0] = self.drone2WP[1]
            self.drone2WP[1] = self.drone2WP[2]
            self.drone2WP[2] = []
            print("toggle state: Drone1", self.toggle_state)
            self.drone[0].setpoint(waypts[-1])
        print("Drone 1 done")
        return
    def targetDrone2(self):
        self.drone[1].hold = 1
        while True:
            # print(self.toggle_state, self.drone2WP[0], 'from drone2')
            if len(self.drone2WP[0]) > 0:
                print(self.toggle_state, self.drone2WP[0], 'from drone2')
                for wayPt in self.drone2WP[0]:
                    self.drone[1].setpoint(wayPt)
                    print("waypt reached: Drone2", wayPt, self.toggle_state)
                    sleep(1)
                self.toggle_state = 1
                print("toggle state: Drone 2", self.toggle_state)
                self.drone[1].setpoint(self.drone2WP[0][-1])
            sleep(1.3)

    def toggle(self, idx):
        if self.drone[idx].hold == 1:
            self.drone[idx].hold = 0
        else:
            self.drone[idx].hold = 1

    def call(self):
        while True:
            # print(self.toggle_state, 'from call')
            if self.toggle_state == 1:
                print('toggled by call')
                self.toggle_state = 0
                self.toggle(0)
                self.toggle(1)
            sleep(1)


In [2]:
BaseManager.register('testClass', testClass)
manager = BaseManager()
manager.start()
myClass = manager.testClass()
p1 = mp.Process(target=myClass.targetDrone1)
p2 = mp.Process(target=myClass.targetDrone2)
p3 = mp.Process(target=myClass.call)
p1.start()
p2.start()
p3.start()
p1.join()

[array([[ 0.8 ,  0.41,  0.5 ],
       [ 0.84, -0.31,  0.5 ]]), array([[ 0.84 , -0.31 ,  0.5  ],
       [ 0.575, -0.31 ,  0.5  ],
       [ 0.31 , -0.31 ,  0.5  ],
       [ 0.045, -0.31 ,  0.5  ],
       [-0.22 , -0.31 ,  0.5  ],
       [-0.485, -0.31 ,  0.5  ],
       [-0.75 , -0.31 ,  0.5  ]]), array([[-0.75, -0.31,  0.5 ],
       [-0.74,  0.07,  0.5 ],
       [-0.73,  0.45,  0.5 ]]), array([[-0.73      ,  0.45      ,  0.5       ],
       [-0.475     ,  0.44333333,  0.5       ],
       [-0.22      ,  0.43666667,  0.5       ],
       [ 0.035     ,  0.43      ,  0.5       ],
       [ 0.29      ,  0.42333333,  0.5       ],
       [ 0.545     ,  0.41666667,  0.5       ],
       [ 0.8       ,  0.41      ,  0.5       ]])]
[[], [[-10, -10, -100], [10, 10, 100]], []]
waypt reached: Drone1 [0.8  0.41 0.5 ] 0
waypt reached: Drone1 [ 0.84 -0.31  0.5 ] 0
toggle state: Drone1 1
1 [[-10, -10, -100], [10, 10, 100]] toggled by callfrom drone2

waypt reached: Drone2 [-10, -10, -100] 0
waypt reached: Dr

In [2]:
manager = mp.Manager()
myClass = manager.list()
myClass=  manager.Namespace()
myClass.shared_variable = testClass()

In [3]:
process1 = mp.Process(target = myClass.shared_variable.targetDrone1)
process2 = mp.Process(target = myClass.shared_variable.targetDrone2)
process3 = mp.Process(target = myClass.shared_variable.call)

In [4]:
process1.start()
process2.start()
process3.start()

0 from call
waypt reached: Drone1 [0.8  0.41 0.5 ] 0
0 from call
0 from call
waypt reached: Drone1 [ 0.84 -0.31  0.5 ] 0
toggle state: Drone1 1 [-10, -10, -100]
0 from call
0 from call
waypt reached: Drone1 [ 0.84 -0.31  0.5 ] 1
0 from call
waypt reached: Drone1 [ 0.575 -0.31   0.5  ] 1
0 from call
waypt reached: Drone1 [ 0.31 -0.31  0.5 ] 1
0 from call
0waypt reached: Drone1  from call[ 0.045 -0.31   0.5  ]
 1
0 from call
waypt reached: Drone1 [-0.22 -0.31  0.5 ] 1
0 from call
waypt reached: Drone1 [-0.485 -0.31   0.5  ] 1
0 from call
waypt reached: Drone1 [-0.75 -0.31  0.5 ] 1
toggle state: Drone1 1 [array([0.8 , 0.41, 0.5 ]), array([ 0.84, -0.31,  0.5 ])]
0 from call
0 from call
waypt reached: Drone1 [-0.75 -0.31  0.5 ] 1
0 from call
0 from call
waypt reached: Drone1 [-0.74  0.07  0.5 ] 1
0 from call
waypt reached: Drone1 [-0.73  0.45  0.5 ] 1
toggle state: Drone1 1 [array([ 0.84, -0.31,  0.5 ]), array([ 0.575, -0.31 ,  0.5  ]), array([ 0.31, -0.31,  0.5 ]), array([ 0.045, -0.31 ,  